In [1]:
import keras
keras.__version__

Using TensorFlow backend.
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sasaki/.local/lib/python3.6/site-packages/tensorflow/python

'2.3.1'

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

606208/600901 [==============================] - 1s 1us/step
Corpus length: 600893


In [4]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

# 2020.02.24 bug-fix
#for i in range(0, len(text) - maxlen, step):
for i in range(0, len(text) - maxlen - 1, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [5]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [6]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1

Epoch 1/1
200278/200278 [==============================] - 237s 1ms/step - loss: 1.9684
--- Generating with seed: "eir exoticism and hidden permanency, were to
depreciate thei"
------ temperature: 0.2
eir exoticism and hidden permanency, were to
depreciate their the soul one one one which and and the bean the father and the conscience of the desire of the desire of the paral one and and in the conscience and the desire of the believe and the pare and the dore the self that is a say of the mast the such and the desire of the soul and the love that in the desire of the desire of the desire of the desire the way one one and and the conscience of the parati
------ temperature: 0.5
 desire the way one one and and the conscience of the parations of the toor conscience of the besure of the desire of man of the profess that the self end one for who wore and the through form and the thought and the quire of the fat to which sense and is in the most pasions of the desire our pasure of t

and involvation of woriight of reisenward woman"
------ temperature: 1.2
be morality
and involvation of woriight of reisenward woman" and the ratuse veverblocee: aodh--as "gresdive divides that one?--but had colsrutiones genium, man, to himself uptories consequence of a down to the, hol,
i? a staurxing after sympathytes of the most crent oby olimentume; say 
i
as consequave.

shur: that innesenveng of bad
bitses has vreass cale)r. dosterior, herkeverner a
classety? theirageuls to "ma.rtory that is oechial same that hundet when 
epoch 5
Epoch 1/1
200278/200278 [==============================] - 227s 1ms/step - loss: 1.4527
--- Generating with seed: "ny more than it was
the french revolution,--the event on acc"
------ temperature: 0.2
ny more than it was
the french revolution,--the event on account of the sund; the thing and the strength and the contemption of the present and the world and the same of the soul of the developed and an all the strength of the god the more to the strength o

en be saint in the same an existence of the period in the ast-andestraccipitically and consiiled philosophers of the spiritor of feeling
was a mo
after; as
which raction. a phill af all bofow i forms of "nothy and besime of med to the elogy of the is aray, much, bioblens shockem."). ther-one with it refines" sainy.=--and formerly accustoas arching manfulness therever time" to dlalty of being and an is bountle; who "very decires fang deed af
the most man, c
------ temperature: 1.2
n is bountle; who "very decires fang deed af
the most man, comsed sainle," sinted a thinker who european instincts has doesle vent oven brought--and moreibhish...tor. slevil
gorty" withvely in the thing that are it mudnes the love. in everythinggliarewitw matke,
philosophical gaodle problem, licas,
light," and the end. an away
upon the highest to pave
bittonous). the arbwacosouther
among to oueiny is assistual faining to de a; "they may with things. whise 
epoch 9
Epoch 1/1
200278/200278 [=====================

 of the surpresfuct of the consequently that the surplaed of intellect in the serious and highly for the most man subjection and sympathy for the very reason and of a regard to the something man to see any strong and to the misunderstand into the surpluted that deception of a character and scarrised to the sortists of the
existence of the surpluted in the surprise the surpresoric of life, and intenrious man and respect reference,
and come must be in the su
------ temperature: 1.0
nrious man and respect reference,
and come must be in the suserudeb; what she is assume--seliction, for the amougism change,
the surprento scientific make into the su,humdos to it. "i can produced maddm and understands?


t
a tfrakly ins; surpllixs. not taken inheses can aspant on a nongumatic
cortrut toar; our glod, he it, of lett his wagners, be sick
was eyes
sen, if such
a ard of the unposition, however,  alled starr
i ackneathy
iny something of god, to
------ temperature: 1.2
, however,  alled starr
i ackn

to disturb the conscience, a lofty and the spiritual and such a present of the spiritual and supposing the spiritual and the spiritual and and present and present and desire and the spiritual and the order of the comparis of the present of the most different that the spiritual and into the spiritual and such a present the most and the most strength in the comparis of the most different and science to the deep of the comparis of the
------ temperature: 0.5
ost different and science to the deep of the comparis of the most a comparis of the most the free
spirits and as a such a present itself and a false and regard and spirit. a poidn are make the adorable souls, in the great sake of the present desires a primardly a present and as fear, and self great fashing of the most conduct and also the spiritual power to all the experience of old privileged to the soul, and in the commander in my course, and come and co
------ temperature: 1.0
the soul, and in the commander in my course, and come a

...

present god, no longingish, an idance, pareter prined and
new immense eskessing ever's lataiges art of healt nothing their go god being an accound of science, mine. he wishes and consequently.--that
in something
in the has peince must darle with the
blund and the inicion of italh: who beling as 
------ temperature: 1.2
arle with the
blund and the inicion of italh: who beling as reveale, and deccics.

227tard!"--anrespequions over the ial, they have nide--this widlureured,
-one the, egots right: doy, and that the longcern as socialismanigorumling and thew conbipauition of things, living causa beginnient worth, generally of the ulthilies. with "inling thirst courhest for what
also a life as it with a must findiled
bothry
you hfoulls this man"-feels enf
: but from shirm of.
epoch 41
Epoch 1/1
200278/200278 [==============================] - 226s 1ms/step - loss: 1.2681
--- Generating with seed: "manner. they all pose as though their real opinions had
been"
------ temperature: 0.2
manner. 

 consideration of the most and consider to the spirit and conceance, and is a god the stand of danger, and there is not decoud comes to the same the fact of the interpretation of the development of them with the same the individual more expressing of the baded them, these so distrust the speciality of music and more self-deat the conception of the faithed the most fantary that the mere in the considen the saparity of an end of the measure. the second the s
------ temperature: 1.0
iden the saparity of an end of the measure. the second the sexual perhaps, vulit that nature--ha: we remain therein--a philosopher and can attentsy, it is perhaps a friends or which is new
enough to case it is as the form of virtue emple
refere the ocdemy, and they indispersuffignismen circum more furnea-gulfuthorous, origioar and not he that her merely yet a colour instincts, in any
higher happing has groper
under those every one attain simplis teption, i
------ temperature: 1.2
g has groper
under those every

himself, following, of course, the taste of his predispension of the same the words, and the scientific and soul and all the soul souls and the spirit and successful and most successful and most spirit and science, the spirits and depthsss of the spirit and all the subjections of the propess of the present of the same the spirit with the strength of the state of the present conditions of the scientific conditions of the scientific waffich an
------ temperature: 0.5
ns of the scientific conditions of the scientific waffich and contradict as the former and suffering, with the speciation of the the subject to the soul of the
innatious and sufficiently understand and stronger, that the most artists of all alloward in pleasure, not himself for a perficted of the boidly and "nature of the suffering conscience, and create have been good in the respect to order to
the art
of the spirit in who has to the desire the subjection
------ temperature: 1.0
he art
of the spirit in who has to the desire

and "termilaritly bad worldly eye seek ows uatrful, bring conscious because "evendsfulur by remunced him
taderdment t
epoch 52
Epoch 1/1
200278/200278 [==============================] - 225s 1ms/step - loss: 1.2518
--- Generating with seed: "decides in favor of the
strongest--as the phrase has it, alt"
------ temperature: 0.2
decides in favor of the
strongest--as the phrase has it, although the fact of the moral philosophers, and the subtlest of the delusion of the belief in the problem of the struggle is not on the greatest and the experience of the words, and the sense of the sense of the germans words, and what is a man of the fact of the fact of the conduct of the moral delights of the stronger of the stronger of the greeks of the subtle and the subtle to the subtle to the 
------ temperature: 0.5
he greeks of the subtle and the subtle to the subtle to the constaration of the world and act for the begins of the german still ancient conditions and even the world and and interpretati

supering. but mark sun, and subter and "longy; couuning,
admiration.


1lävet over envortion:
he does, what whanto "are
fomationd; dangers ham all noist differently spectrful flagely of
modern irroded, same mono-trea,
under their minglinity increteruation
of opporations, it is asijes:--couragion
for a new, your sefecy the fine philosophans disposing, however, than on
the goos and thingslofitating mangervary
knowledge, the weye i
epoch 56
Epoch 1/1
200278/200278 [==============================] - 227s 1ms/step - loss: 1.2466
--- Generating with seed: "nother, who has the inheritance of a diversified descent in "
------ temperature: 0.2
nother, who has the inheritance of a diversified descent in the command the subjugation of the sense of the world of the sense of the sense of the constraint of the contrary of the most surceed of the sense of the moral powerful and the sense of the heart of the sense of the same the self sense to the sense of the sense of the respect of the sense of the 